In [ ]:
import pandas as pd #este es el codigo bueno
import mysql.connector
from mysql.connector import errorcode
from sqlalchemy import create_engine

import urllib.parse

import sys
import os 
import pandas as pd

from dotenv import load_dotenv

# ---<---<--<--@    hasta aquí nuestras librerías   @-->-->--->---

In [ ]:
# Ya tenemos los datos, ahora necesitamos el puente a MySQL 

load_dotenv()

def conectar():
    try:
        cnx = mysql.connector.connect(
            user='root',
            password=os.getenv("MYSQL_PASSWORD"),
            host='127.0.0.1',
            database='MusicStreamStupendo',
            auth_plugin='mysql_native_password')
        
        print('Llamada (( ☎ )) a SQL\n')

    except mysql.connector.Error as err:
        print(f'Error: {err}')

    return cnx

# Llama a la función
cnx = conectar()
print('SQL ha contestado (( ☎ )) está listo para trabajar\n')

Conexión a "MusicStreamStupendo" ok.
Cargando DataFrame: artistas_lastfm_combinados_final.csv 
✅ DataFrame cargado. Tamaño: (552, 5)
Enviando datos a SQL
🎉Datos cargados a la tabla 'info_artistas' en MySQL.


In [ ]:


ruta_archivo = '../csv/datos_artistas_lastfm.csv' 
# -----------------------------------------------



def cargar_csv_a_sql(ruta_csv, nombre_tabla):
    # 1. Validar la conexión (Si falla, el script se detiene)
    if not conectar_y_validar():
        print("No se pudo conectar a MySQL.")
        sys.exit(1)
        
    # 2. Cargar el DataFrame
    print(f"Cargando DataFrame: {os.path.basename(ruta_csv)} ")
    try:
        # Usamos 'utf-8' para manejar correctamente acentos
        df = pd.read_csv(ruta_csv, encoding='utf-8') 
        print(f'✅ DataFrame cargado. Tamaño: {df.shape}')
        
    except Exception as e:
        # Si la lectura falla, lanzamos error y terminamos
        raise RuntimeError(f"Error al leer el archivo CSV: {e}") 

    # 3. Carga a SQL con SQLAlchemy
    print("Enviando datos a SQL")
    
    # Codificamos la contraseña para el URL de conexión
    db_password_encoded = urllib.parse.quote_plus(db_password)
    
    # Creamos el motor de conexión
    engine_url = f"mysql+mysqlconnector://{db_user}:{db_password_encoded}@{db_host}/{database_name}"
    
    try:
        engine = create_engine(engine_url)
        
        # Cargar el DataFrame a la tabla SQL
        df.to_sql(
            name=nombre_tabla, 
            con=engine, 
            if_exists='replace', # Reemplaza la tabla si ya existe
            index=False
        )
        print(f"🎉Datos cargados a la tabla '{nombre_tabla}' en MySQL.")
    except Exception as e:
        raise RuntimeError(f"Error al cargar los datos a MySQL: {e}")

# --- USO FINAL DEL CÓDIGO ---
try:
    cargar_csv_a_sql(ruta_archivo, nombre_tabla)
except Exception as e:
    print(f"FATAL ERROR EN EL PROCESO: {e}")